In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_ex').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv('Ecommerce_Customers.csv',
                         inferSchema=True,header=True)

In [6]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [7]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [8]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [10]:
for item in data.head(2)[1]:
    print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [11]:
from pyspark.ml.linalg import Vectors

In [12]:
from pyspark.ml.feature import VectorAssembler

In [13]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [14]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
                                       'Time on App',
                                       'Time on Website',
                                       'Length of Membership']
                             ,outputCol = 'features')

In [15]:
output = assembler.transform(data)

In [16]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [17]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [18]:
final_data = output.select('features','Yearly Amount Spent')

In [19]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [20]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [21]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                353|
|   mean|  500.7613183514882|
| stddev|  81.38781996241447|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [22]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                147|
|   mean|  495.8385969470751|
| stddev|  74.25296890867897|
|    min|   266.086340948469|
|    max|  689.7876041747194|
+-------+-------------------+



In [23]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [24]:
lrModel = lr.fit(train_data)

In [25]:
test_results = lrModel.evaluate(test_data)

In [26]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -7.364074518017617|
|-0.8867260770246617|
|-19.554190277414932|
| -5.160613463145353|
|  6.434886283757123|
| -21.72070489594182|
| -4.662922187369077|
| 2.0434007345182863|
| 1.6276343929354766|
|  -8.51827016111281|
| -6.960771199835335|
| -17.81274059932116|
| -7.447282296678509|
|-3.4364959332453395|
|  2.712269319769632|
| -2.448770497705823|
| -11.69752718573875|
| -6.527593773352407|
|-17.739921923568374|
| 12.044971915346537|
+-------------------+
only showing top 20 rows



In [27]:
test_results.rootMeanSquaredError

10.313059291190141

In [28]:
test_results.r2

0.9805771989331084

In [29]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [30]:
unlabeled_data = test_data.select('features')

In [31]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.4925366965402...|
|[30.5743636841713...|
|[30.8162006488763...|
|[30.8364326747734...|
|[30.9716756438877...|
|[31.1239743499119...|
|[31.2681042107507...|
|[31.3091926408918...|
|[31.4459724827577...|
|[31.4474464941278...|
|[31.5147378578019...|
|[31.5702008293202...|
|[31.7207699002873...|
|[31.7216523605090...|
|[31.7366356860502...|
|[31.8627411090001...|
|[31.8648325480987...|
|[31.8745516945853...|
|[31.9048571310136...|
|[31.9096268275227...|
+--------------------+
only showing top 20 rows



In [32]:
predictions = lrModel.transform(unlabeled_data)

In [33]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...|289.83532023793214|
|[30.5743636841713...| 442.9511398350903|
|[30.8162006488763...|285.64053122588393|
|[30.8364326747734...|472.66251389013496|
|[30.9716756438877...| 488.2037234731356|
|[31.1239743499119...| 508.6677587357076|
|[31.2681042107507...|  428.133455361193|
|[31.3091926408918...|430.67731710541534|
|[31.4459724827577...| 483.2493305421931|
|[31.4474464941278...| 427.1210122563368|
|[31.5147378578019...|496.77325919629675|
|[31.5702008293202...|  563.758232740726|
|[31.7207699002873...| 546.2222157747015|
|[31.7216523605090...|  351.213422565118|
|[31.7366356860502...| 494.2211769357623|
|[31.8627411090001...| 558.7469116717525|
|[31.8648325480987...| 451.5888076625524|
|[31.8745516945853...| 398.8128380196199|
|[31.9048571310136...| 491.6897793463845|
|[31.9096268275227...| 551.4010637578926|
+--------------------+------------